# CasADiの使い方
## はじめに
最適制御問題を数値的に解くためのツールとして，CasADiを用いる．使い方を勉強する．
### 参考
- [CasADi's documentation](https://web.casadi.org/docs/)
- [CasADi's GitHub Repository](https://github.com/casadi/casadi)

### Install
```bash
pip install casadi
```

## Documentについて
以下，ドキュメントを見てコードの書き方を勉強する．
### 1. Introduction
- CasADiは，gradient-basedの数値最適化 (特に最適制御に重点をおいている) のためのツール．
- ユーザに最適制御問題を入力させ，その解を返すわけではなく，最適制御問題のソルバーを少しのプログラミング作業で効率的に実装するために使用できる "building block" を提供するもの．

### 2. Obtaining and installing
- open-source tool
- C++で書かれている．front-endにはPythonもある．

### 3. Symbolic framework
- CasADiのcoreは， "self-contained symbolic framework" であること．
- MATLABのような "everything-is-a-matrix" 構文で記号式を構築できる．
- vectors: n-by-1 matrices, scalars: 1-by-1 matrices

#### 3.1. The `sx` symbols

In [1]:
###### Packages ######
from casadi import * 
######################

変数`x`を定義する．

In [5]:
###### Define variable x ######
x = SX.sym("x")
print(x)
###############################

x


上記のコードで定義した`x`は，1-by-1 matrix (scalar) である．"x"は，表示名であり，識別子ではない．

`SX.sym`に追加の引数を指定してvector-valued, matrix-valuedの変数を定義できる．

In [6]:
y = SX.sym('y', 5)
z = SX.sym('Z', 4, 2)
print(y)
print(z)

[y_0, y_1, y_2, y_3, y_4]

[[Z_0, Z_4], 
 [Z_1, Z_5], 
 [Z_2, Z_6], 
 [Z_3, Z_7]]


Remark
- `SX.sym`は，`SX`インスタンスを返す (静的な) 関数である．
- 変数が宣言されると，式を直感的な方法で形成できる．

In [8]:
f = x**2 + 10 
f = sqrt(f)
print(f)

sqrt((sq(x)+10))


symbolic primitivesがなくても`SX`インスタンスを定義できる．

- `B1 = SX.zeros(4, 5)`: すべての要素がzerosのdense 4-by-5 empty matrix
- `B2 = SX(4, 5)`: すべての要素がzerosのsparse 4-by-5 empty matrix
- `B4 = SX.eye(4)`: 4-by-4の単位行列

In [9]:
B1 = SX.zeros(4, 5)
B2 = SX(4, 5)
B4 = SX.eye(4)

print(B1)
print(B2)
print(B4)

@1=0, 
[[@1, @1, @1, @1, @1], 
 [@1, @1, @1, @1, @1], 
 [@1, @1, @1, @1, @1], 
 [@1, @1, @1, @1, @1]]

[[00, 00, 00, 00, 00], 
 [00, 00, 00, 00, 00], 
 [00, 00, 00, 00, 00], 
 [00, 00, 00, 00, 00]]
@1=1, 
[[@1, 00, 00, 00], 
 [00, @1, 00, 00], 
 [00, 00, @1, 00], 
 [00, 00, 00, @1]]


Note

- sparse matrixとdense matrixの違いは，structural zerosとactual zeros．
- `00`: structural zero
- `0`: actual zero

`SX`の表現を以下にまとめる．

- `SX.sym(name, n, m)`: n-by-mのsymbolic primitiveを定義する．
- `SX.zeros(n, m)`: すべての要素がzerosのn-by-m dense matrixを定義する．
- `SX(n, m)`: すべての要素がstructual zerosのn-by-m sparse matrixを定義する．
- `SX.ones(n, m)`: すべての要素が1のn-by-m dense matrixを定義する．
- `SX.eye(n)`: 対角成分が1で，その他の成分がstructural zerosのn-by-n対角行列を定義する．
- `SX(scalar_type)`: argumentで与えられた値を持つscalar (1-by-1 matrix) を定義する．
- `SX(matrix_type)`: NumpyやScipyのmatrixで与えられたnumerical matrixを定義する．
- `repmat(v, n, m)`: vを縦にn回，横にm回繰り返した表現 (e.g. `repmat(SX(3), 2, 1)`: すべての要素が3の2-by-1 matrix)．
- (Python only) `SX(list)`: list内の要素を成分とする縦ベクトル (n-by-1 matrix) を定義する (e.g. `SX([1, 2, 3, 4])`)．
- (Python only) `SX(list of list)`: list内の要素を成分とするdense matrixを定義する (e.g. `SX([[1, 2], [3, 4]])`)．

#### 3.2. DM
- `SX`と似ているが，nonzero elementsが数値であり，symbolic expressioinsではないという点で`SX`と異なる．構文はほぼ同じ．
- `DM`は主に，CasADiに行列を格納するため，また，関数の入出力として使われる．

In [10]:
C = DM(2, 3)
print(C)

C_dense = C.full()
print(C_dense)
from numpy import array 
C_dense = array(C)  # equivalent 
print(C_dense)

C_sparse = C.sparse()
print(C_sparse)
from scipy.sparse import csc_matrix 
C_sparse = csc_matrix(C)    # equivalent
print(C_sparse)


[[00, 00, 00], 
 [00, 00, 00]]
[[0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]]




#### 3.3. The `MX` symbolics
以下の`SX`の演算を考える．

In [11]:
x = SX.sym('x', 2, 2)
y = SX.sym('y')
f = 3*x + y 
print(f)
print(f.shape)

@1=3, 
[[((@1*x_0)+y), ((@1*x_2)+y)], 
 [((@1*x_1)+y), ((@1*x_3)+y)]]
(2, 2)


上記の操作の出力は2-by-2の行列となっている．乗算と加算が要素ごとに実行され，結果行列のeach entryに対してSX型の新しい式が作成された．

より一般的なmatrix expression typeの`MX`を導入する．`MX`の基本演算はスカラーの単項演算 ($$\mathbb{R} \to \mathbb{R}$$) や二項演算 ($$\mathbb{R} \times \mathbb{R} \to \mathbb{R}$$) に制限されていない．

`MX`では，一般的な複数のsparse-matrix入力で複数のsparse-matrix出力の関数の操作ができる ($$\mathbb{R}^{n_1 \times m_1} \times \cdots \times \mathbb{R}^{n_N \times m_N} \to \mathbb{R}^{p_1 \times q_1} \times \cdots \times \mathbb{R}^{p_M \times q_M}$$)．

上記の`SX`の演算を`MX`で書く．

In [12]:
x = MX.sym('x', 2, 2)
y = MX.sym('y')
f = 3*x + y 
print(f)
print(f.shape)

((3*x)+y)
(2, 2)


`MX`でも`SX`と同様に，要素を取得すること，セットすることができる．

In [13]:
x = MX.sym('x', 2, 2)
print(x[0, 0])

x[0]


出力は，xの最初のstructurally non-zeroの要素に等しい式 (index 0 (最初の) matrixの位置)

In [14]:
x = MX.sym('x', 2)
A = MX(2, 2)
A[0,0] = x[0]
A[1,1] = x[0] + x[1]
print('A:', A)

A: (project((zeros(2x2,1nz)[0] = x[0]))[1] = (x[0]+x[1]))


#### 3.4. Mixing `SX` and `MX`
- `SX`オブジェクトと`MX`オブジェクトに対する乗算操作や，同じexpression graphで2つのオブジェクトを混合することはできない．
- `MX` graphには，`SX` expressionsで定義された関数への呼び出しを含めることができる．
- `SX` expressionsは低レベルの操作で使用することを目的としている．
- `MX` expressionsはNLPの制約関数などの作成に使用できる．

#### 3.5. The `Sparsity` class
CasADiでは，matricesをcompressed column storage (CCS) formatに格納できる．CCS formatは，sparse matricesのstandard format．線形代数の演算を可能にする．

CCS formatでは，sparsity patternはdimensions (rowsとcolumnsの数) と2つのベクトルを使ってdecodeされる．

CasADiのSparsity patternは，Sparsity classのインスタンスとして格納される．